### module 

In [17]:
import mysql.connector
from datetime import datetime
import mysql.connector
import pandas as pd
import warnings
from tqdm.notebook import tqdm

### function 

1. get code names 

In [5]:
def get_table_names(host, user, password, database):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query to retrieve table names
        query = f"SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{database}'"
        cursor.execute(query)

        # Fetch all the rows
        tables = cursor.fetchall()

        # Print table names
        # for (table_name,) in tables:
        #     print(table_name)

    except mysql.connector.Error as error:
        print(f"Error: {error}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            return tables

2. get data in schema 

In [6]:
def table_to_dataframe(host, user, password, database, table_name):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Use backticks around the table name in the query
        query = f"SELECT * FROM `{table_name}`"

        # Use pandas to read the query into a DataFrame
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            df = pd.read_sql(query, conn)

        return df

    except mysql.connector.Error as error:
        print(f"Error: {error}")
        return None
    finally:
        if conn.is_connected():
            conn.close()

### Implement 

1. get ticker code 

In [7]:
# databaseA = "A" + datetime.now().strftime("%Y%m%d") 
databaseA = 'a20240112'
password = '93150lbm!!'

tables = get_table_names('127.0.0.1', 'root', password, databaseA)

tables = [code for (code,) in tables]
print(tables)

['000440', '005290', '009620', '009730', '013990', '021080', '024060', '025320', '027360', '027830', '029480', '033540', '036540', '036630', '038870', '041020', '041190', '041460', '047310', '048770', '049080', '050890', '053050', '056080', '057680', '058610', '059100', '060250', '060280', '060310', '064260', '065350', '065450', '066790', '068760', '072950', '080220', '080530', '085670', '086960', '088800', '090150', '090360', '090710', '094170', '094480', '096630', '098460', '099440', '104480', '108490', '115160', '117730', '129890', '136480', '142210', '203650', '205100', '205470', '207760', '208340', '214270', '214680', '215100', '218150', '227100', '228760', '232140', '241820', '253590', '254120', '264850', '270520', '270660', '277070', '290690', '291230', '293580', '299900', '306620', '307930', '317830', '319400', '321260', '321370', '348340', '356680', '380540', '382800', '383930', '388790', '391710', '403870', '417200', '419050', '419270', '450520', '452190', '452300', '900250']

test one ticker 

In [19]:
code = tables[1]

df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)
df['체결시간'] = pd.to_datetime(df['체결시간'])
df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
df = df.sort_values(by='체결시간', ignore_index=True)

# print(df)
print(code)
for i in tqdm(range(len(df))):
    
    start_idx = i
    start_time = df.loc[start_idx,'체결시간']
    
    interval_time = start_time + pd.Timedelta(minutes=2)
    end_index = df[df['체결시간'] < interval_time].index.max()
    
    
    interval_data = df.loc[start_idx:end_index]
    min_price = interval_data['현재가'].min()
    max_price = interval_data['현재가'].max()
    fluctuation = ((max_price - min_price) /min_price ) * 100
    
    
    start_price =  interval_data['현재가'].iloc[0]
    end_price = interval_data['현재가'].iloc[-1]
    
    if fluctuation > 5 and end_price > start_price : 
        print(start_time)

005290


  0%|          | 0/22920 [00:00<?, ?it/s]

test many tickers 

In [20]:
signal = {}

for code in tqdm(tables) : 
    
    print(code)
    df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)
    df['체결시간'] = pd.to_datetime(df['체결시간'])
    df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
    df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
    df = df.sort_values(by='체결시간', ignore_index=True)

    for i in range(len(df)):
        
        start_idx = i
        start_time = df.loc[start_idx,'체결시간']
        
        interval_time = start_time + pd.Timedelta(minutes=2)
        end_index = df[df['체결시간'] < interval_time].index.max()
        
        
        interval_data = df.loc[start_idx:end_index]
        min_price = interval_data['현재가'].min()
        max_price = interval_data['현재가'].max()
        fluctuation = ((max_price - min_price) /min_price ) * 100
        
        
        start_price =  interval_data['현재가'].iloc[0]
        end_price = interval_data['현재가'].iloc[-1]
        
        if fluctuation > 5 and end_price > start_price : 
            print(start_time)
            if code not in signal.keys():
                    signal[code] = []
            signal[code].append(start_time)

  0%|          | 0/100 [00:00<?, ?it/s]

000440
005290
009620
009730
013990
021080
024060
025320
027360
027830
029480
033540
036540
036630
038870
041020
041190
041460
047310
048770
2024-01-12 12:21:23.363635
2024-01-12 12:21:24.691172
2024-01-12 12:21:26.257253
2024-01-12 12:21:26.364625
2024-01-12 12:21:28.261541
2024-01-12 12:21:30.874054
2024-01-12 12:21:39.173236
2024-01-12 12:21:40.069343
2024-01-12 12:21:40.069343
2024-01-12 12:21:40.069343
2024-01-12 12:21:40.069850
2024-01-12 12:21:40.069850
2024-01-12 12:21:45.774970
2024-01-12 12:21:48.066524
2024-01-12 12:21:55.188594
2024-01-12 12:21:57.372392
2024-01-12 12:21:57.372392
2024-01-12 12:21:57.652869
2024-01-12 12:22:11.779245
2024-01-12 12:22:15.863164
2024-01-12 12:22:16.261029
2024-01-12 12:22:16.804706
2024-01-12 12:22:16.804706
2024-01-12 12:22:16.804706
2024-01-12 12:22:17.256429
2024-01-12 12:22:17.256429
2024-01-12 12:22:17.456353
2024-01-12 12:22:17.456353
2024-01-12 12:22:17.457349
2024-01-12 12:22:17.457349
2024-01-12 12:22:17.457349
2024-01-12 12:22:17.457

In [21]:
signal

{'048770': [Timestamp('2024-01-12 12:21:23.363635'),
  Timestamp('2024-01-12 12:21:24.691172'),
  Timestamp('2024-01-12 12:21:26.257253'),
  Timestamp('2024-01-12 12:21:26.364625'),
  Timestamp('2024-01-12 12:21:28.261541'),
  Timestamp('2024-01-12 12:21:30.874054'),
  Timestamp('2024-01-12 12:21:39.173236'),
  Timestamp('2024-01-12 12:21:40.069343'),
  Timestamp('2024-01-12 12:21:40.069343'),
  Timestamp('2024-01-12 12:21:40.069343'),
  Timestamp('2024-01-12 12:21:40.069850'),
  Timestamp('2024-01-12 12:21:40.069850'),
  Timestamp('2024-01-12 12:21:45.774970'),
  Timestamp('2024-01-12 12:21:48.066524'),
  Timestamp('2024-01-12 12:21:55.188594'),
  Timestamp('2024-01-12 12:21:57.372392'),
  Timestamp('2024-01-12 12:21:57.372392'),
  Timestamp('2024-01-12 12:21:57.652869'),
  Timestamp('2024-01-12 12:22:11.779245'),
  Timestamp('2024-01-12 12:22:15.863164'),
  Timestamp('2024-01-12 12:22:16.261029'),
  Timestamp('2024-01-12 12:22:16.804706'),
  Timestamp('2024-01-12 12:22:16.804706'),
 